## Business Problem: One of the most important problems in e-commerce is the correct calculation of points. given to aftermarket products. Solving this problem means providing more. Customer satisfaction for e-commerce site, product prominence for sellers and a seamless shopping experience for buyers. Another problem is correct ordering. The prominence of misleading comments among the comments made on the products financial loss and loss of customers. That's why amazon company wants to rank reviews, and when ordering these reviews, they want to rank them according to their sentiment (positive / negative). In solving these 2 basic problems, e-commerce site and sellers will increase their sales, customers hassle-free purchasing journey

In [5]:
!pip install chart_studio
!pip install TextBlob
!pip install plotly
%pip install wordcloud
!pip install cufflinks
pip install nbformat
pip install wordcloud
pip install seaborn



Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
import nltk
import re
from textblob import TextBlob
from wordcloud import wordcloud
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
%matplotlib inline
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True)
cf.go_offline();
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")
pd.set_option('display.max_columns', None)

In [9]:
dataraw = pd.read_csv("/Users/heyminhminh/Downloads/amazon_reviews.csv")

In [10]:
data = dataraw.copy()

In [11]:
data = data.sort_values("wilson_lower_bound", ascending=False)
data.drop('Unnamed: 0', inplace = True, axis = 1)
data.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5.0,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,2013-01-05,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5.0,I have tested dozens of SDHC and micro-SDHC ca...,2012-09-26,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1.0,NOTE: please read the last update (scroll to ...,2013-05-08,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1.0,"If your card gets hot enough to be painful, it...",2012-02-09,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5.0,Sandisk announcement of the first 128GB micro ...,2014-07-03,158,45,4,49,41,0.918367,0.808109


In [13]:
def missing_values_analysis(data):
    na_columns_ = [col for col in data.columns if data[col].isnull().sum() > 0]
    n_miss = data[na_columns_].isnull().sum().sort_values(ascending=True)
    ratio_ = (data[na_columns_].isnull().sum() / data.shape[0] * 100).sort_values(ascending=True)
    missing_data = pd.concat([n_miss, np.round(ratio_, 2)], axis=1, keys=['Total Missing Values', 'Ratio'])
    missing_data = pd.DataFrame(missing_data)
    return missing_data

def check_dataframe(data, head=5, tail = 5):
    
    print(" SHAPE ".center(82,'~'))
    print('Rows: {}'.format(data.shape[0]))
    print('Columns: {}'.format(data.shape[1]))
    print(" TYPES ".center(82,'~'))
    print(data.dtypes)
    print("".center(82,'~'))
    print(missing_values_analysis(data))
    print(' DUPLICATED VALUES '.center(83,'~'))
    print(data.duplicated().sum())
numerical_data = data.select_dtypes(include=np.number)
quantiles = numerical_data.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T
print(quantiles)

check_dataframe(data)

                       0.00  0.05   0.50        0.95       0.99         1.00
overall                 1.0   2.0    5.0    5.000000    5.00000     5.000000
day_diff                1.0  98.0  431.0  748.000000  943.00000  1064.000000
helpful_yes             0.0   0.0    0.0    1.000000    3.00000  1952.000000
helpful_no              0.0   0.0    0.0    0.000000    2.00000   183.000000
total_vote              0.0   0.0    0.0    1.000000    4.00000  2020.000000
score_pos_neg_diff   -130.0   0.0    0.0    1.000000    2.00000  1884.000000
score_average_rating    0.0   0.0    0.0    1.000000    1.00000     1.000000
wilson_lower_bound      0.0   0.0    0.0    0.206549    0.34238     0.957544
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ SHAPE ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Rows: 4915
Columns: 11
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ TYPES ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
reviewerName             object
overall                 float64
reviewText               object
reviewTime            

In [14]:
def check_class(dataframe):
    nunique_data = pd.DataFrame({'Variable': dataframe.columns,
                                 'Classes': [dataframe[i].nunique() \
                                             for i in dataframe.columns]})
    nunique_data = nunique_data.sort_values('Classes', ascending=False)
    nunique_data = nunique_data.reset_index(drop=True)
    return nunique_data

check_class(data)


,Variable,Classes
0,reviewText,4912
1,reviewerName,4594
2,reviewTime,690
3,day_diff,690
4,wilson_lower_bound,40
5,score_average_rating,28
6,score_pos_neg_diff,27
7,total_vote,26
8,helpful_yes,23
9,helpful_no,17


In [15]:
# categorical variable analysis ---> overall

constraints = ['#581845','#C70039','#2E4053','#1ABC9C','#7F8C8D']

def categorical_variable_summary(data, column_name):
    fig = make_subplots(rows=1,cols=2,
                        subplot_titles=('Countplot','Percentages'),
                        specs=[[{"type": "xy"}, {'type':'domain'}]])

    fig.add_trace(go.Bar( y = data[column_name].value_counts().values.tolist(), 
                          x = [str(i) for i in data[column_name].value_counts().index], 
                          text = data[column_name].value_counts().values.tolist(),
                          textfont = dict(size=15),
                          name = column_name,
                          textposition = 'auto',
                          showlegend=False,
                          marker=dict(color = constraints,
                                      line=dict(color='#DBE6EC',
                                                width=1))),
                  row = 1, col = 1)
    
    fig.add_trace(go.Pie(labels= data[column_name].value_counts().keys(),
                         values= data[column_name].value_counts().values,
                         textfont = dict(size = 20),
                         textposition='auto',
                         showlegend = False,
                         name = column_name,
                         marker=dict(colors=constraints)),
                  row = 1, col = 2)
    
    fig.update_layout(title={'text': column_name,
                             'y':0.9,
                             'x':0.5,
                             'xanchor': 'center',
                             'yanchor': 'top'},
                      template='plotly_white')
    
    iplot(fig)

In [16]:
categorical_variable_summary(data,'overall')

In [27]:
# Sentiment analysis
# TextBlob Exit will return polarity and subjectivity.
# Polarity indicates your mood, that is, whether it is positive.
# It returns a value between 0 and 1. The closer to 1 the more positive, the closer to 0 the more negative.
# Replace NaN values with an empty string
data['reviewText'] = data['reviewText'].fillna('')
data[['polarity', 'subjectivity']] = data['reviewText'].apply(lambda Text: pd.Series(TextBlob(str(Text)).sentiment))

# Sentiment analysis using VADER Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()
data['vader_scores'] = data['reviewText'].apply(lambda Text: analyzer.polarity_scores(str(Text))['compound'])

# Iterate over the rows of the DataFrame
for index, row in data.iterrows():
    if pd.notnull(row['reviewText']):
        score = SentimentIntensityAnalyzer().polarity_scores(str(row['reviewText']))
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']
    if neg > pos:
        data.loc[index, 'sentiment'] = "Negative"
    elif pos > neg:
        data.loc[index, 'sentiment'] = "Positive"
    else:
        data.loc[index, 'sentiment'] = "neutral"

In [28]:
# 20 Identifying the interpretation, now we can include the positive, negative and neutral status of the comments.

data[data["sentiment"] == "Positive"].sort_values("wilson_lower_bound", ascending=False).head(5)

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound,polarity,subjectivity,vader_scores,sentiment
2031,"Hyoun Kim ""Faluzure""",5.0,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,2013-01-05,702,1952,68,2020,1884,0.966337,0.957544,0.156702,0.558574,0.9975,Positive
3449,NLee the Engineer,5.0,I have tested dozens of SDHC and micro-SDHC ca...,2012-09-26,803,1428,77,1505,1351,0.948837,0.936519,0.091564,0.514756,0.9910,Positive
4212,SkincareCEO,1.0,NOTE: please read the last update (scroll to ...,2013-05-08,579,1568,126,1694,1442,0.925620,0.912139,0.208749,0.501294,0.9995,Positive
317,"Amazon Customer ""Kelly""",1.0,"If your card gets hot enough to be painful, it...",2012-02-09,1033,422,73,495,349,0.852525,0.818577,0.151042,0.470566,0.8473,Positive
4672,Twister,5.0,Sandisk announcement of the first 128GB micro ...,2014-07-03,158,45,4,49,41,0.918367,0.808109,0.164206,0.493237,0.9872,Positive


In [30]:
# Let's see if we have an unbalanced data problem
categorical_variable_summary(data,'sentiment')

In [32]:
# Let's see if there is an imbalance in the scoring?
data.groupby(["sentiment"])[['overall']].mean()

,overall
sentiment,
Negative,3.598450
Positive,4.745468
neutral,4.624161
